In [ ]:
!pip install tinyimagenet
!pip install torchvision
!pip install seaborn
!pip install torchmetrics
!pip install poutyne

from tinyimagenet import TinyImageNet
from pathlib import Path
import torch
from torchvision import models

In [ ]:


weights = models.EfficientNet_V2_S_Weights.IMAGENET1K_V1
base_model = models.efficientnet_v2_s(weights=weights)
# freeze base model
for param in base_model.parameters():
    param.requires_grad = False

# add new classification head
tinyimagenet_classes = 200
base_model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2,inplace=True),
    torch.nn.Linear(1280, tinyimagenet_classes),
)

# add normalization transforms
model = torch.nn.Sequential(
    weights.transforms(),
    base_model,
    
)
print(base_model)

In [ ]:
import poutyne


device = torch.device("cuda")
def get_dataset(split): return TinyImageNet(Path("~/.torchvision/tinyimagenet/"),split=split,imagenet_idx=False)

train_dataset = get_dataset("train")
val_dataset = get_dataset("val")


trainer = poutyne.Model(
        model,
        'adam',
        'cross_entropy',
        batch_metrics=['accuracy',poutyne.TopKAccuracy(5)],
        epoch_metrics=['f1'],
        device=device
)


trainer.fit_dataset(train_dataset,valid_dataset=val_dataset,batch_size=32,epochs=5)